In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import anuga
import parameters as p
%matplotlib inline

# Allow inline jshtml animations
from matplotlib import rc
rc('animation', html='jshtml')

/tmp/ipykernel_35679/1801362053.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Cargado de datos

In [4]:
''' PARÁMETROS '''

INPUT_FOLDER = os.path.abspath("input")
CICLO = 9
SECTORES = [5, 6]

folder_ciclo = os.path.join(INPUT_FOLDER, f"ciclo_{CICLO}")

In [5]:
''' DESCARGAS '''
suffix_input = '_'.join([str(s) for s in SECTORES])
df_descargas = pd.read_csv(os.path.join(folder_ciclo, "input_sectores_{}.csv".format(suffix_input)), parse_dates=['fecha'])

In [6]:
''' RUTAS '''
ruta_topografia = os.path.join(folder_ciclo, "DEM_04.02.2023_10x10m.asc")
ruta_mascara_tranque = p.RUTA_MASK_TRANQUE
ruta_sector_5 = os.path.join(folder_ciclo, "poligono_sector_5.csv")
ruta_interior = os.path.join(folder_ciclo, "zona_interior_sector_5.csv")
ruta_extension_sector_5 = os.path.join(folder_ciclo, "extension_sector_5.csv")

In [7]:
''' MALLADO '''
res_region = 100
res_interior = 50

# Construcción input

In [12]:
df_input = df_descargas[df_descargas.sector == 5]
df_input = df_input[df_input.fecha == df_input.fecha.min()]

In [13]:
df_input

,fecha,sector,id_punto,coordenada_x,coordenada_y,angulo_polar,tms,tasa_diaria,radio_canaleta
153,2023-03-02,5,29,489215.492474,7.297292e+06,2.425051,22137.258627,0.961598,1.6345
154,2023-03-02,5,32,489082.721325,7.296706e+06,2.536601,22137.258627,0.961598,1.4370
155,2023-03-02,5,33,488926.557163,7.296312e+06,2.907850,22137.258627,0.961598,1.6430
156,2023-03-02,5,37,488770.083732,7.295805e+06,2.377836,44274.517254,1.923196,1.3145
